In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [ ]:
matches18 = pd.read_csv('135_2018.csv', sep=';')
matches19 = pd.read_csv('135_2019.csv', sep=';')
matches20 = pd.read_csv('135_2020.csv', sep=';')
matches21 = pd.read_csv('135_2021.csv', sep=';')
matches22 = pd.read_csv('135_2022.csv', sep=';')

In [25]:
matches19['away_form_5'].dtype

dtype('O')

In [14]:
matches19[matches19['round']==20]

,match_date,round,match_id,home_team_id,home_team,away_team_id,away_team,home_goals,away_goals,home_pred,...,comparison_def,comparison_poisson,comparison_h2h,comparison_goals,comparison_total,home_bet,draw_bet,away_bet,under_bet,over_bet
189,2020-01-18 15:00:00+01:00,20,232720,487,Lazio,498,Sampdoria,5,1,45%,...,57%,86%,93%,76%,73.5%,1.39,4.78,7.95,2.34,1.59
190,2020-01-18 18:00:00+01:00,20,232722,488,Sassuolo,503,Torino,2,1,10%,...,46%,46%,15%,33%,33.3%,2.61,3.33,2.72,2.01,1.80
191,2020-01-18 20:45:00+01:00,20,232715,492,Napoli,502,Fiorentina,0,2,45%,...,50%,52%,62%,45%,50.5%,1.62,4.09,5.33,2.17,1.68
192,2020-01-19 12:30:00+01:00,20,232723,489,AC Milan,494,Udinese,3,2,45%,...,42%,61%,62%,56%,50.5%,1.54,3.95,6.77,1.99,1.81
193,2020-01-19 15:00:00+01:00,20,232714,518,Brescia,490,Cagliari,2,2,45%,...,55%,29%,50%,50%,54.0%,2.47,3.51,2.76,1.94,1.87
194,2020-01-19 15:00:00+01:00,20,232716,500,Bologna,504,Verona,1,1,45%,...,56%,49%,71%,71%,56.8%,1.94,3.48,4.02,1.98,1.83
195,2020-01-19 15:00:00+01:00,20,232717,867,Lecce,505,Inter,1,1,0%,...,21%,17%,29%,23%,21.8%,9.73,5.58,1.30,2.72,1.45
196,2020-01-19 18:00:00+01:00,20,232719,495,Genoa,497,AS Roma,1,3,10%,...,38%,22%,25%,44%,33.8%,4.06,3.49,1.93,1.95,1.85
197,2020-01-19 20:45:00+01:00,20,232718,496,Juventus,523,Parma,2,1,45%,...,54%,74%,71%,72%,65.5%,1.22,6.46,13.18,2.65,1.47
198,2020-01-20 20:45:00+01:00,20,232721,499,Atalanta,493,Spal,1,2,45%,...,62%,87%,62%,50%,70.2%,1.19,7.07,13.82,3.03,1.37


In [20]:
matches22['match_date'] = pd.to_datetime(matches22['match_date'], utc=True)
matches22['target'] = (matches22['home_goals']-matches22['away_goals'] > 0).astype('int')

,match_date,round,match_id,home_team_id,home_team,away_team_id,away_team,home_goals,away_goals,home_pred,...,comparison_poisson,comparison_h2h,comparison_goals,comparison_total,home_bet,draw_bet,away_bet,under_bet,over_bet,target
0,2022-08-13 16:30:00+00:00,1,881780,489,AC Milan,494,Udinese,4.0,2.0,45%,...,0%,75%,57%,66.0%,1.55,4.75,6.85,2.19,1.83,1
1,2022-08-13 16:30:00+00:00,1,881781,498,Sampdoria,499,Atalanta,0.0,2.0,0%,...,0%,20%,25%,22.5%,5.15,4.20,1.62,2.16,1.74,0
2,2022-08-13 18:45:00+00:00,1,881782,867,Lecce,505,Inter,1.0,2.0,0%,...,0%,29%,23%,26.0%,10.25,5.70,1.29,2.35,1.54,0
3,2022-08-13 18:45:00+00:00,1,881783,1579,Monza,503,Torino,1.0,2.0,33%,...,0%,0%,0%,0%,2.53,3.32,2.83,1.74,2.16,0
4,2022-08-14 16:30:00+00:00,1,881784,502,Fiorentina,520,Cremonese,3.0,2.0,33%,...,0%,0%,0%,0%,1.53,4.35,5.95,2.13,1.76,1


In [21]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [43]:
train = matches22[matches22['match_date'] < '2022-11-01']
test = matches22[matches22['match_date'] > '2022-11-01']
test.shape

(45, 79)

In [44]:
predictors = ['home_team_id', 'away_team_id', 'home_goals_for_avg_h', 'home_goals_against_avg_h', 'away_goals_for_avg_a', 'away_goals_against_avg_a']

In [45]:
rf.fit(train[predictors], train['target'])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [46]:
preds = rf.predict(test[predictors])

In [48]:
acc = accuracy_score(test['target'], preds)
acc

0.5555555555555556